In [41]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import keras
import pandas as pd

이미지가 0, 1, 2, ... , 255 까지 값을 가지는 배열이므로 255 로 스케일링 해서 0 ~ 1 사이 값을 가지고 하는 것임. 

이렇게 해야 활성화함수 및 오류역전파 알고리즘이 잘 동작함.

![ex_screenshot](img/Train_Valid_Test.jpg)

In [49]:
# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'warehouse/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode="categorical",
        shuffle=True)

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
        'warehouse/handwriting_shape/valid',
        target_size=(24, 24),    
        batch_size=1,
        class_mode="categorical",
        shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/handwriting_shape/test',
        target_size = (24, 24),    
        batch_size = 1,
        class_mode = None,
        shuffle = False)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Found 13 images belonging to 1 classes.


24 * 24 이미지가 3 * 3 사이즈 32개 컨볼루션 레이어를 통과하면 (22, 22, 32) 가 됨.

이게 다시 같은 사이즈의 64개 컨볼루션 레이어를 통과하면 (20, 20, 64) 가 됨.

이게 (2, 2) 즉 50% 로 줄이는 맥스풀링 레이어를 통과하면 (10, 10, 64) 가 됨.

플래튼 레이어를 통과하면 10 * 10 * 64 = 6400 개 배열이 됨.

In [50]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

데이터 생성 시 배치사이즈를 3으로 했으므로 학습 시 한 세대 당 스텝 개수는 15로 한다. 그래야 샘플 개수 3 * 15 = 45 가 됨.

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [51]:
# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        generator=train_generator,
        steps_per_epoch=15,        
        validation_data=valid_generator,
        validation_steps=5,
        epochs=5
)

Epoch 1/5
15/15 [==============================] - 1s 57ms/step - loss: 1.1462 - acc: 0.6667 - val_loss: 0.5262 - val_acc: 1.0000
Epoch 2/5
15/15 [==============================] - 0s 18ms/step - loss: 0.1862 - acc: 0.9778 - val_loss: 0.4760 - val_acc: 0.8000
Epoch 3/5
15/15 [==============================] - 0s 17ms/step - loss: 0.0205 - acc: 1.0000 - val_loss: 0.0128 - val_acc: 1.0000
Epoch 4/5
15/15 [==============================] - 0s 18ms/step - loss: 6.9533e-04 - acc: 1.0000 - val_loss: 2.0942e-04 - val_acc: 1.0000
Epoch 5/5
15/15 [==============================] - 0s 18ms/step - loss: 3.5494e-04 - acc: 1.0000 - val_loss: 0.1108 - val_acc: 1.0000


In [52]:
# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(generator=valid_generator, steps=15)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
print(scores)

-- Evaluate --
acc: 100.00%
[0.009606291445258345, 1.0]


flow_from_directory 의 파라미터 중 batch_size 는 디폴트값이 32 임. 

모든 샘플에 대해서 하나의 예측만 하고 싶으면 predict_generator 의 steps 를 샘플개수/batch_size 로 넣으면 됨

target_size 는 원본 이미지가 최종적으로 리사이징 될 크기.
color mode 는 흑백이나 그레이스케일인 경우는 grayscale 로 넣는다.
batch_size 는 배치 당 generator 로 부터 산출되는 이미지 갯수
class_mode 는 2 class 문제일 경우에는 "binary" 로 Autoencoder(입력과 출력이 같아야 하는) 시스템일 경우는 "input" 으로 설정함
shuffle 은 산출되는 이미지를 shuffle 하고 싶을 때는 True 로 한다
seed 는 Random seed for applying random image augmentation and shuffling the order of the image 라고 함.

train_generator = train_datagen.flow_from_directory(
    directory=r"./train/",
    target_size=(224, 224),
    color_mode="rgb"
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [53]:
test_generator.reset()
filenames = test_generator.filenames
nb_samples = len(filenames)
pred=model.predict_generator(test_generator,steps=nb_samples)

In [54]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [55]:
print(filenames)
print(predictions)

['Test_folder\\circle016.png', 'Test_folder\\circle017.png', 'Test_folder\\circle018.png', 'Test_folder\\rectangle007.png', 'Test_folder\\rectangle008.png', 'Test_folder\\rectangle009.png', 'Test_folder\\rectangle012.png', 'Test_folder\\rectangle013.png', 'Test_folder\\rectangle014.png', 'Test_folder\\rectangle019.png', 'Test_folder\\rectangle020.png', 'Test_folder\\triangle017.png', 'Test_folder\\triangle020.png']
['circle', 'circle', 'circle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'triangle', 'triangle']


In [32]:
filenames = valid_generator.filenames
nb_samples = len(filenames)
print(filenames)

['circle\\circle016.png', 'circle\\circle017.png', 'circle\\circle018.png', 'circle\\circle019.png', 'circle\\circle020.png', 'rectangle\\rectangle016.png', 'rectangle\\rectangle017.png', 'rectangle\\rectangle018.png', 'rectangle\\rectangle019.png', 'rectangle\\rectangle020.png', 'triangle\\triangle016.png', 'triangle\\triangle017.png', 'triangle\\triangle018.png', 'triangle\\triangle019.png', 'triangle\\triangle020.png']


test_generator 의 batch_size 는 train_generator 나 evaluate_generator 의 batch_size 와 조금 다르다.
test set 의 전체 이미지 개수를 나누기 위한 어떤 숫자인데 정확하게 한번만 샘플링 되어야 한다. 잘 모르겠으면 1 로 둔다.
class_mode 는 None 으로 둔다. 오직 이미지만을 Return 하기 위해...

test_generator = test_datagen.flow_from_directory(
    directory=r"./test/",
    target_size=(224, 224),
    color_mode="rgb"
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

argmax 함수의 axis 는 0=열(column), 1=행(row), 2=면(page) 의 순으로 가장 높은 숫자의 위치를 알려주는 파라미터임.

In [23]:
# 6. 모델 사용하기
print("-- Predict --")
print(valid_generator.filenames)
valid_generator.reset()
print(valid_generator.filenames)
#output = model.predict_generator(val_generator, verbose=1)
output = model.predict_generator(valid_generator, nb_samples)
# output = model.predict_generator(val_generator, steps=nb_samples)
output2 = np.argmax(output, axis=1)
#np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
#print(val_generator.class_indices)
#print(val_generator.classes)
print(valid_generator.filenames)
#print("%s: " %(model.metrics_names[1]))
#print(output)
print(output2)
label_map = train_generator.class_indices
#print(label_map)
label_map2 = dict((v,k) for k,v in label_map.items()) #flip k,v
print('label_map2 = ' + str(label_map2))
output3 = [label_map2[k] for k in output2]
print(output3)

filenames=valid_generator.filenames
#results=pd.DataFrame({"Filename":filenames,"Predictions":output3})
#results.to_csv("results.csv",index=False)

-- Predict --
['circle\\circle016.png', 'circle\\circle017.png', 'circle\\circle018.png', 'circle\\circle019.png', 'circle\\circle020.png', 'rectangle\\rectangle016.png', 'rectangle\\rectangle017.png', 'rectangle\\rectangle018.png', 'rectangle\\rectangle019.png', 'rectangle\\rectangle020.png', 'triangle\\triangle016.png', 'triangle\\triangle017.png', 'triangle\\triangle018.png', 'triangle\\triangle019.png', 'triangle\\triangle020.png']
['circle\\circle016.png', 'circle\\circle017.png', 'circle\\circle018.png', 'circle\\circle019.png', 'circle\\circle020.png', 'rectangle\\rectangle016.png', 'rectangle\\rectangle017.png', 'rectangle\\rectangle018.png', 'rectangle\\rectangle019.png', 'rectangle\\rectangle020.png', 'triangle\\triangle016.png', 'triangle\\triangle017.png', 'triangle\\triangle018.png', 'triangle\\triangle019.png', 'triangle\\triangle020.png']
['circle\\circle016.png', 'circle\\circle017.png', 'circle\\circle018.png', 'circle\\circle019.png', 'circle\\circle020.png', 'rectang